In [6]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
fw = open('Transaction-Record.txt','w')

In [2]:
# here, coin number is saved as float (complying with code from github, while contradicts white paper)
class Smartcoin(object):
	def __init__(self, name='kennycoin', reservetokenName='aCoin', initCRR=0.5, initPrice=1, initIssueNum=100):
		self._Name = name
		self._ReservetokenName = reservetokenName
		self._CRR = float(initCRR)
		self._Price = float(initPrice)
		self._Supply = float(initIssueNum)
		self._ReserveBalance = float(initCRR * initIssueNum)
		self._budget = float(0)     

	def printInfo(self):
		print '---------'
		print 'NAME:', self._Name, '| RESERVE NAME:', self._ReservetokenName, 'CRR:', self._CRR
		print 'PRICE:',self._Price
		print 'SUPPLY:', self._Supply, '| RESERVE BALANCE:', self._ReserveBalance
		print 'BUDGET:', self._budget

	def saveInfo(self, fw):
		fw.write('---------\n')
		fw.write('NAME: '+str(self._Name)+' | RESERVE NAME: '+str(self._ReservetokenName) + ' | CRR: '+str(self._CRR)+'\n')
		fw.write('PRICE: '+str(self._Price))
		fw.write('SUPPLY: '+str(self._Supply)+' | RESERVE BALANCE: '+str(self._ReserveBalance)+'\n')
        
	def updatePrice(self, reserveBalance, supply, CRR):
		newPrice = reserveBalance/(supply * CRR)
		return newPrice
    
	def setCRR(self, newCRR = 0.5):
		oldCRR = self._CRR
		self._CRR = newCRR
		print 'CRR', oldCRR, '->', newCRR

	def getPrice(self):
		return self._Price

	def purchasing(self, convertIntoNum=0):
		# ETH be convert into BNT
		issuedtokenNum = self._Supply * (((self._ReserveBalance + convertIntoNum)/self._ReserveBalance)**(self._CRR) - 1)
		self._Supply = self._Supply + issuedtokenNum
		self._ReserveBalance = self._ReserveBalance + convertIntoNum
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		increaseRatio = (self._Price - oldPrice)/oldPrice
# 		print '&&&&&&&&&&'
# 		print round(convertIntoNum,2), self._ReservetokenName+" be converted into as", round(issuedtokenNum,2), self._Name
# 		print "Current price of "+self._Name+" is", self._Price, "with", round(increaseRatio*100,4), "% increasing."
		self._budget = self._budget + self._Price*self._Supply - oldPrice*(self._Supply - issuedtokenNum)
		return issuedtokenNum

	def destroying(self, convertOutNum=0):
		# BNT be converted out to ETH
		destroyedtokenNum = convertOutNum      
		reserveReceivedNum = self._ReserveBalance*(1 - ((self._Supply - convertOutNum)/self._Supply)**(1/self._CRR))
		self._Supply = self._Supply - destroyedtokenNum
		self._ReserveBalance = self._ReserveBalance - reserveReceivedNum
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		decreaseRatio = (oldPrice-self._Price)/oldPrice
# 		print '**********'
# 		print round(convertOutNum,2), self._Name+" be converted out as", round(reserveReceivedNum,2), self._ReservetokenName
# 		print "Current price of "+self._Name+" is", self._Price, "with", round(decreaseRatio*100,4), "% decreasing."
		self._budget = self._budget + self._Price*self._Supply - oldPrice*(self._Supply + destroyedtokenNum)
		return reserveReceivedNum

In [3]:
class Customers(object):
    def __init__(self, smartToken, ownedSmartTokens = float(0), reserveValue = float(500),expectedPrice = 0):
        # _smartToken is the token customers want to buy -- SmartToken()
        self._smartToken = smartToken
        # _ownedSmartToken refers to smart token's number customers have
        self._ownedSmartToken = float(ownedSmartTokens)
        # _smartValue is the value of ownedSmartTokens -- currentPrice * owned#
        self._smartValue = smartToken.getPrice() * ownedSmartTokens
        self._reserveValue = float(reserveValue)
        # _ownedvalue = _reserveValue + _smartValue
        self._ownedvalue = float(reserveValue)
        # gain or lose money comparing to the original state
        self._budget = float(0)
        # expectedPrice denotes the how much money customers are willing to finish the transaction 
        self._expectedPrice = expectedPrice
        
    def printinfo(self):
        # since new reserve will be converted into or out the kenny coin, the budget could change
        # when print info, update the budget
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._smartToken._Price * self._ownedSmartToken + self._reserveValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        print '------'
        print 'smartToken Name:', self._smartToken._Name
        print 'ownedvalue:', self._ownedvalue, 'budget:', self._budget
        
    def getReserveValue(self):
        return self._reserveValue
    def getownedSmartToken(self):
        return self._ownedSmartToken
    
    # add customer's reserve amount
    def addReserve(self, addAmount):
        self._reserveValue = self._reserveValue + addAmount
        self._ownedvalue = self._smartToken._Price * self._ownedSmartToken + self._reserveValue
    
    # returned expected Price
    def getExpectedPrice(self):
        return self._expectedPrice
        
    # change the expected price
    def changeExpectedPrice(self, newExpectedPrice):
        self._expectedPrice = newExpectedPrice
        
    def purchase(self, reserveTokenNumber):
        oldPrice = self._smartToken._Price
        if reserveTokenNumber > self._reserveValue:
            print 'Invalid Operation in Purchase'
            return
        # smartToken.purchasing means being converted
        issuedSmartToken = self._smartToken.purchasing(reserveTokenNumber)
        newPrice = self._smartToken._Price      
        self._ownedSmartToken = self._ownedSmartToken + issuedSmartToken
        self._smartValue = self._ownedSmartToken * newPrice
        self._reserveValue = self._reserveValue - reserveTokenNumber
        # in fact, in this case ownedValue and budget actually does not change at all
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._reserveValue + self._smartValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        
        
    def destroy(self, smartTokenNumber):
        oldPrice = self._smartToken._Price
        if self._ownedSmartToken < smartTokenNumber:
            print 'Invalid Operation in Destroy'
            return
        self._ownedSmartToken = self._ownedSmartToken - smartTokenNumber
        # smartToken.purchasing means being purchased
        ReceivedToken = self._smartToken.destroying(smartTokenNumber)
        newPrice = self._smartToken._Price
        self._smartValue = self._ownedSmartToken*newPrice
        self._reserveValue = self._reserveValue + ReceivedToken
        # In fact, in this case ownedValue actually does not change at all
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._reserveValue + self._smartValue
        self._budget = self._budget + self._ownedvalue - oldownedValue

In [10]:
# issue a new smart token
initIssue = 300000
CRR = 0.2
initTransaction = initIssue * (1-CRR)
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=initIssue)

In [19]:
# issue a new smart token
initIssue = 200000000
CRR = 0.2
initTransaction = initIssue * (1-CRR)
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=initIssue)
'''
init properties: 
We have #TimeRound round,
In each time round, #custNum customers come in, 
   with original reserve: #custOriginalReserve
   with original smarttokens: #custOriginalSmartTokens
'''
TimeRound = 100
custNum = 1000
sigma = 0.1
custOriginalReserve = 100
custOriginalSmartTokens = 100
'''
prevent the init money exhausts the balance, 
here we already set initIssue SmartToken to 300000,
which actually is hard to exhaust,
also, we should consider the situation that the init issued smarttokens is comparably small,
      or the init smarttokens is held in customers in a large amount
'''
if TimeRound * custNum * custOriginalSmartTokens > initTransaction:
    print 'WARNING, too many init smart tokens from customers'

# PriceTracker records the change of the price
PriceTracker = []
# failedTracker records the transaction failure
failedBuy_rateTracker = []
failedSell_rateTracker = []
failed_rateTracker = []

j=0
while j < TimeRound:
    # failed num denotes the number of transactions failed in each time round
    failed_num = 0
    failed_buyNum = 0
    failed_sellNum = 0
    buyNum = 0
    sellNum = 0
    
    i=0
    custlist = []
    CurrentPrice = KennyCoin.getPrice()
    OldPrice = CurrentPrice
    # set the expectation of initPrice (first we fix it), Should it change with time?
    initPrice = CurrentPrice
    ''' 
    First, we initialize the customers who come in at #j TimeRound.
    We assume in each TimeRound, #custNum customers come in, whose number is gaussian distributed with custExpectedPrice
    We assume when price is mu -- initPrice, customers' number is the largest
    Since we get customers #custNum involved, the normal should be sampled by custNum.
    '''
    mu = initPrice
#     np.random.seed(0)
    custExpectedPrice = np.random.normal(mu, sigma, custNum)
    while i < custNum:
        Joe = Customers(smartToken = KennyCoin, ownedSmartTokens = custOriginalSmartTokens, reserveValue = custOriginalReserve, expectedPrice = custExpectedPrice[i])
        custlist.append(Joe)
        i = i + 1
    for Joe in custlist:
        if Joe.getExpectedPrice() > OldPrice:
            buyNum = buyNum + 1
            if Joe.getExpectedPrice() >= KennyCoin.getPrice():
                # Joe purchases successfully
                custReserve = Joe.getReserveValue()
                randomBuy = random.uniform(0,custReserve)
                Joe.purchase(reserveTokenNumber = randomBuy)
            else:
                # failed in buying
                failed_buyNum = failed_buyNum + 1
        elif Joe.getExpectedPrice() < OldPrice:
            sellNum = sellNum + 1
            if Joe.getExpectedPrice() <= KennyCoin.getPrice():
                # Joe sells successfully
                custSmartToken_Num = Joe.getownedSmartToken()
                randomSell = random.uniform(0,custSmartToken_Num)
                Joe.destroy(smartTokenNumber = randomSell)
            else:
                # failed in selling
                failed_sellNum = failed_sellNum + 1
        else:
            '''
            This case means:
              Joe.getExpectedPrice() == OldPrice,
            Then we need to throw a coin to determine whether to buy or sell,
              half - half possibility
            '''
            seed = random.randint(0, 100)
            if seed < 50:
                buyNum = buyNum + 1
                if Joe.getExpectedPrice() >= KennyCoin.getPrice():
                    # Joe purchases successfully
                    custReserve = Joe.getReserveValue()
                    randomBuy = random.uniform(0,custReserve)
                    Joe.purchase(reserveTokenNumber = randomBuy)
                else:
                    # failed in buying
                    failed_buyNum = failed_buyNum + 1
            else:
                sellNum = sellNum + 1
                if Joe.getExpectedPrice() <= KennyCoin.getPrice():
                    # Joe sells successfully
                    custSmartToken_Num = Joe.getownedSmartToken()
                    randomSell = random.uniform(0,custSmartToken_Num)
                    Joe.destroy(smartTokenNumber = randomSell)
                else:
                    # failed in selling
                    failed_sellNum = failed_sellNum + 1
        CurrentPrice = KennyCoin.getPrice()
        PriceTracker.append((CurrentPrice,j))
    print 'The',j,'round, BuyNum:',buyNum,'SellNum:',sellNum
    failed_num = failed_buyNum + failed_sellNum
    failedBuy_rateTracker.append((float(failed_buyNum) / buyNum , j))
    failedSell_rateTracker.append((float(failed_sellNum) / sellNum , j))
    failed_rateTracker.append((float(failed_num) / (buyNum+sellNum) , j))
    fw.write('After Round '+ str(j) + '\n')
    KennyCoin.saveInfo(fw)
    j = j + 1

1.09529253058
The 0 round, BuyNum: 482 SellNum: 518
1.03077876565
The 1 round, BuyNum: 520 SellNum: 480
1.04803209817
The 2 round, BuyNum: 517 SellNum: 483
1.05022180712
The 3 round, BuyNum: 490 SellNum: 510
1.16712626369
The 4 round, BuyNum: 516 SellNum: 484
1.08451132878
The 5 round, BuyNum: 501 SellNum: 499
0.946831364854
The 6 round, BuyNum: 496 SellNum: 504
1.08935629441
The 7 round, BuyNum: 506 SellNum: 494
0.90784141791
The 8 round, BuyNum: 511 SellNum: 489
0.835171154667
The 9 round, BuyNum: 493 SellNum: 507
0.993119732157
The 10 round, BuyNum: 493 SellNum: 507
0.985230468526
The 11 round, BuyNum: 487 SellNum: 513
1.21079308093
The 12 round, BuyNum: 528 SellNum: 472
1.07805220275
The 13 round, BuyNum: 532 SellNum: 468
0.932672684696
The 14 round, BuyNum: 515 SellNum: 485
0.999267570261
The 15 round, BuyNum: 488 SellNum: 512
1.08392336766
The 16 round, BuyNum: 474 SellNum: 526
0.965853276554
The 17 round, BuyNum: 512 SellNum: 488
1.03959894415
The 18 round, BuyNum: 520 SellNum: 

In [45]:
KennyCoin.printInfo()

---------
NAME: Kenny | RESERVE NAME: ETH CRR: 0.2
PRICE: 2.07637843008
SUPPLY: 360120.531037 | RESERVE BALANCE: 149549.300575
BUDGET: 447746.502874


In [54]:
print failedBuy_rateTracker[6] ,failedSell_rateTracker[6], failed_rateTracker[6]

(0.38461538461538464, 6) (0.5, 6) (0.44, 6)


In [20]:
j = 0
failedBuyRate = []
failedSellRate = []
failedRate = []
myX = []
while j < TimeRound:
    myX.append(j)
    failedRate.append(failed_rateTracker[j][0])
    failedBuyRate.append(failedBuy_rateTracker[j][0])
    failedSellRate.append(failedSell_rateTracker[j][0])
    j = j + 1
x = np.asarray(myX)
y1 = np.asarray(failedRate)
y2 = np.asarray(failedBuyRate)
y3 = np.asarray(failedSellRate)
plt.plot(x,y1,'o-',color = 'navy',alpha = 0.8)
plt.title('Failure Rate Change For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Failure Rate of Transaction', fontsize = 15)
plt.savefig('Failure/FailureRate-All.png', bbox_inches='tight')
plt.close()
plt.plot(x,y2,'o-',color = 'navy',alpha = 0.8)
plt.title('Buy Failure Rate Change For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Failure Rate of Buy Transaction', fontsize = 15)
plt.savefig('Failure/FailureRate-Buy.png', bbox_inches='tight')
plt.close()
plt.plot(x,y3,'o-',color = 'navy',alpha = 0.8)
plt.title('Sell Failure Rate Change For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Failure Rate of Sell Transaction', fontsize = 15)
plt.savefig('Failure/FailureRate-Sell.png', bbox_inches='tight')
plt.close()

In [50]:
print PriceTracker[101]

(3.7057318690059224, 1)


In [21]:
j = 0
PriceAllRound = []
my_Xall = []
while j < TimeRound:
    myX = []
    Price_eachRound = []
    i = 0
    while i < custNum:
        myX.append(i+1)
        Price_eachRound.append(PriceTracker[i+j*custNum][0])
        i = i + 1
    PriceAllRound.append(Price_eachRound[custNum-1])
    # here to prevent the Price_eachRound plot cost too much time
    if TimeRound <= 50:
        x = np.asarray(myX)
        y = np.asarray(Price_eachRound)
        plt.plot(x,y,'o-',color = 'navy',alpha = 0.8)
        plt.title('Price Change in Round \"'+str(j)+'\"',fontsize = 25)
        plt.xlabel('Customer #',fontsize = 15)
        plt.ylabel('Price of Smart Token', fontsize = 15)
        plt.savefig('Price/PriceRound/Price_inRound'+str(j)+'.png', bbox_inches='tight')
        plt.close()
        print 'Round '+str(j)+' Over...'
    my_Xall.append(j)
    j = j + 1
allX = np.asarray(my_Xall)
ally = np.asarray(PriceAllRound)
plt.plot(allX,ally,'o-',color = 'navy',alpha = 0.8)
plt.title('Price Change For All Round',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Price of Smart Token', fontsize = 15)
plt.savefig('Price/Price_Change.png', bbox_inches='tight')
plt.close()

In [4]:
# example of using smartcoin class
BNTCoin = Smartcoin(name='BNT',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)
BNTCoin.printInfo()
BNTCoin.purchasing(300)
BNTCoin.printInfo()
BNTCoin.purchasing(700)
BNTCoin.printInfo()
BNTCoin.destroying(1302)
BNTCoin.printInfo()
BNTCoin.purchasing(100)
BNTCoin.printInfo()

---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.0
SUPPLY: 300000.0 | RESERVE BALANCE: 60000.0
BUDGET: 0.0
*********
300.0 ETH be converted into as 299.4 BNT
Current prize of BNT is 1.00399800399 with 0.3998 % increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.00399800399
SUPPLY: 300299.401794 | RESERVE BALANCE: 60300.0
BUDGET: 1500.0
*********
700.0 ETH be converted into as 694.0 BNT
Current prize of BNT is 1.01331125792 with 0.9276 % increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.01331125792
SUPPLY: 300993.399232 | RESERVE BALANCE: 61000.0
BUDGET: 5000.0
*********
1302.0 BNT be converted out as 1307.97 ETH
Current prize of BNT is 0.995891667702 with 1.7191 % decreasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 0.995891667702
SUPPLY: 299691.399232 | RESERVE BALANCE: 59692.0334754
BUDGET: -1539.83262295
*********
100.0 ETH be converted into as 100.35 BNT
Current prize of BNT is 0.997226150558 with 0.134 % increasing.
---

In [12]:
# test for customers class
Alice = Customers(smartToken=KennyCoin)
print 'Current Price of KennyCoin: ', round(KennyCoin.getPrice(),4)
Alice.purchase(100)
print 'Current Price of KennyCoin: ', round(KennyCoin.getPrice(),4)

Current Price of KennyCoin:  1.0067
*********
100.0 ETH be converted into as 99.27 Kenny
Current prize of Kenny is 1.00799203183 with 0.1322 % increasing.
Current Price of KennyCoin:  1.008
